### In Depth Analysis - Modeling & Hyper parameter tuning

In [1]:
%matplotlib inline
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
sns.set()

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier

from scipy import stats

from sklearn.preprocessing import MinMaxScaler
from sklearn.impute import SimpleImputer

from keras.layers import Input, Dense
from keras.models import Model, Sequential
from keras import regularizers
from keras import optimizers
import tensorflow as tf
from keras import backend as K
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()

from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from skopt import BayesSearchCV

from sklearn.metrics import accuracy_score,classification_report, roc_auc_score

Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
##all features
application_train = pd.read_csv('new_application_train_merged.csv', index_col=None, engine='python')
application_test = pd.read_csv('new_application_test_merged.csv', index_col=None, engine='python')


In [3]:
#top 20 features
new_application_train = pd.read_csv('application_train_for_ML.csv', index_col=None, engine='python')
new_application_test = pd.read_csv('application_test_for_ML.csv', index_col=None, engine='python')

## Feature Imputing & Feature Scaling 


In [4]:
new_application_train.shape

(43845, 21)

In [5]:
imputer = SimpleImputer(strategy="median")
scaler = MinMaxScaler(feature_range = (0, 1))

In [6]:
imputer.fit(new_application_train.drop(['TARGET'], axis=1))


SimpleImputer(strategy='median')

In [7]:
train_transformed = imputer.transform(new_application_train.drop(['TARGET'], axis=1))
train_transformed = scaler.fit_transform(train_transformed)


In [8]:
test_transformed = imputer.transform(new_application_test)
test_transformed = scaler.fit_transform(test_transformed)


In [9]:
X_training_set, X_validation_set, y_training_set, y_validation_set = train_test_split(train_transformed, 
                                                                                      new_application_train['TARGET'], test_size=0.33, random_state=42)

In [10]:
def metric_calc(pred):
    print("The accuracy score : ", accuracy_score(y_validation_set,pred))
    print("The classification report is as follows:\n", classification_report(y_validation_set,pred))
    print("Taregt Values:")
    print(pd.DataFrame({'target':pred})['target'].value_counts())
    print("ROC AUC score is: ",roc_auc_score(y_validation_set,pred))

## 1. Logistic Regression

In [11]:
logistic_regressor = LogisticRegression(C = 2)
logistic_regressor.fit(X_training_set,y_training_set)
log_regression_pred = logistic_regressor.predict(X_validation_set)
pd.DataFrame({'target':log_regression_pred})['target'].value_counts()
metric_calc(log_regression_pred)

The accuracy score :  0.6712972562029166
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.67     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.67      0.67      0.67     14469

Taregt Values:
1    8327
0    6142
Name: target, dtype: int64
ROC AUC score is:  0.6658108026204868


## 2. Random Forest 

In [12]:

random_forest = RandomForestClassifier(n_estimators = 500, random_state = 50, verbose = 1)

In [13]:
random_forest.fit(X_training_set,y_training_set)


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:   33.7s finished


RandomForestClassifier(n_estimators=500, random_state=50, verbose=1)

In [14]:
random_forest_pred = random_forest.predict(X_validation_set)
metric_calc(random_forest_pred)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


The accuracy score :  0.6746838067592784
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.67     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.67      0.67      0.67     14469

Taregt Values:
1    8326
0    6143
Name: target, dtype: int64
ROC AUC score is:  0.6692317960672662


[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    3.2s finished


In [15]:
random_forest_pred_test = random_forest.predict_proba(test_transformed)
submission_random_forest = application_test[['SK_ID_CURR']]
submission_random_forest['TARGET'] = random_forest_pred_test[:,1]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 500 out of 500 | elapsed:    7.8s finished
<ipython-input-15-86f9992d35cb>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_random_forest['TARGET'] = random_forest_pred_test[:,1]


## 2.1 Feature importance of random forest classifier


In [16]:
features = list(set(new_application_train.columns).difference({'TARGET'}))

feature_importance_df = pd.DataFrame({'Feature':features,'Importance':random_forest.feature_importances_})
df = feature_importance_df.sort_values(['Importance'],ascending=False).reset_index()
df = df.drop(['index'],axis=1)
df.head()

,Feature,Importance
0,EXT_SOURCE_2,0.176427
1,REGION_RATING_CLIENT,0.162095
2,FLAG_MOBIL,0.123399
3,AMT_CREDIT_x,0.119232
4,NAME_FAMILY_STATUS,0.084473


## 3. Naive Bayes Classifier


In [17]:
bayes_class = GaussianNB()
bayes_class.fit(X_training_set,y_training_set)
bayes_preds = bayes_class.predict(X_validation_set)
metric_calc(bayes_preds)

The accuracy score :  0.6523602183979542
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.62      0.62      0.62      6578
           1       0.68      0.68      0.68      7891

    accuracy                           0.65     14469
   macro avg       0.65      0.65      0.65     14469
weighted avg       0.65      0.65      0.65     14469

Taregt Values:
1    7803
0    6666
Name: target, dtype: int64
ROC AUC score is:  0.6500302040198895


In [18]:
nb_pred_test = bayes_class.predict_proba(test_transformed)
submission_nb = application_test[['SK_ID_CURR']]
submission_nb['TARGET'] = nb_pred_test[:,1]


<ipython-input-18-3115073d959d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_nb['TARGET'] = nb_pred_test[:,1]


## 4 .XG Boost

In [19]:
xgb_classifier = XGBClassifier(objective ='binary:logistic')
xgb_classifier.fit(X_training_set,y_training_set)
xgb_pred = xgb_classifier.predict(X_validation_set)
metric_calc(xgb_pred)

The accuracy score :  0.6703987836063308
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.60      0.62      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.67     14469
   macro avg       0.67      0.66      0.67     14469
weighted avg       0.67      0.67      0.67     14469

Taregt Values:
1    8370
0    6099
Name: target, dtype: int64
ROC AUC score is:  0.6646329460239638


In [20]:
xgb_pred_test = xgb_classifier.predict_proba(test_transformed)
submission_xgb = application_test[['SK_ID_CURR']]
submission_xgb['TARGET'] = xgb_pred_test[:,1]


<ipython-input-20-65d003118767>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_xgb['TARGET'] = xgb_pred_test[:,1]


## 5 . Ensemble Modeling 
https://quantdare.com/what-is-the-difference-between-bagging-and-boosting/

In [21]:
#Voting Approach
def stacked_model(X_training_set):
    """
    This method performs the stacked ensambling of all the models - Random forest, 
    Naive Bayes,Logistic Regression.
    """  
    stacked_predictions = np.array([])

    for element in X_training_set:
         stacked_predictions = np.append(stacked_predictions,stats.mode(element)[0][0])

    return stacked_predictions

Combine all the test results into a multidimensional array to feed into the stacked model

In [22]:
combined_array = (pd.DataFrame({'LR':log_regression_pred,
                                'RF':random_forest_pred,
                                'Bayes':bayes_preds,
                                'XGB':xgb_pred}).values)
stacked_model_pred = stacked_model(combined_array)
metric_calc(stacked_model_pred)

The accuracy score :  0.6767572050590919
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.64      0.66      0.65      6578
           1       0.71      0.69      0.70      7891

    accuracy                           0.68     14469
   macro avg       0.67      0.68      0.68     14469
weighted avg       0.68      0.68      0.68     14469

Taregt Values:
1.0    7634
0.0    6835
Name: target, dtype: int64
ROC AUC score is:  0.6756984867435408


## Observation : Ensemble provides slight better result though not signifacnt improvement 

## 6. Neural Network 

In [23]:
model = Sequential()
new_application_test.shape[1]
model.add(Dense(100, input_dim=new_application_train.drop(['TARGET'], axis=1).shape[1], activation= "relu"))
model.add(Dense(50, activation= "relu"))
model.add(Dense(1,activation= "sigmoid"))
model.summary() #Print model Summary


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 100)               2100      
_________________________________________________________________
dense_1 (Dense)              (None, 50)                5050      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 51        
Total params: 7,201
Trainable params: 7,201
Non-trainable params: 0
_________________________________________________________________


In [24]:
def auroc(y_true, y_pred):
    return tf.py_func(roc_auc_score, (y_true, y_pred), tf.double)

In [25]:
model.compile(loss= "binary_crossentropy" , optimizer="adam", metrics=['accuracy', auroc])
model.fit(X_training_set, y_training_set, epochs=250)


Instructions for updating:
tf.py_func is deprecated in TF V2. Instead, there are two
    options available in V2.
    - tf.py_function takes a python function which manipulates tf eager
    tensors instead of numpy arrays. It's easy to convert a tf eager tensor to
    an ndarray (just call tensor.numpy()) but having access to eager tensors
    means `tf.py_function`s can use accelerators such as GPUs as well as
    being differentiable using a gradient tape.
    - tf.numpy_function maintains the semantics of the deprecated tf.py_func
    (it is not differentiable, and manipulates numpy arrays). It drops the
    stateful argument making all functions stateful.
    
Train on 29376 samples
Epoch 1/250
29376/29376 [==============================] - 2s 75us/sample - loss: 0.6267 - acc: 0.6471 - auroc: 0.7058
Epoch 2/250
29376/29376 [==============================] - 3s 96us/sample - loss: 0.6143 - acc: 0.6630 - auroc: 0.7215
Epoch 3/250
29376/29376 [==============================] - 2s 84us

29376/29376 [==============================] - 3s 90us/sample - loss: 0.5726 - acc: 0.6979 - auroc: 0.7673
Epoch 63/250
29376/29376 [==============================] - 2s 69us/sample - loss: 0.5722 - acc: 0.6981 - auroc: 0.7678
Epoch 64/250
29376/29376 [==============================] - 2s 64us/sample - loss: 0.5705 - acc: 0.6969 - auroc: 0.7686
Epoch 65/250
29376/29376 [==============================] - 2s 70us/sample - loss: 0.5703 - acc: 0.6990 - auroc: 0.76991s - loss
Epoch 66/250
29376/29376 [==============================] - 2s 66us/sample - loss: 0.5694 - acc: 0.6984 - auroc: 0.7688
Epoch 67/250
29376/29376 [==============================] - 2s 72us/sample - loss: 0.5686 - acc: 0.7007 - auroc: 0.7709
Epoch 68/250
29376/29376 [==============================] - 3s 87us/sample - loss: 0.5679 - acc: 0.7015 - auroc: 0.7722
Epoch 69/250
29376/29376 [==============================] - 3s 87us/sample - loss: 0.5667 - acc: 0.7002 - auroc: 0.7728
Epoch 70/250
29376/29376 [==================

Epoch 130/250
29376/29376 [==============================] - 2s 64us/sample - loss: 0.5270 - acc: 0.7286 - auroc: 0.8088
Epoch 131/250
29376/29376 [==============================] - 2s 66us/sample - loss: 0.5246 - acc: 0.7310 - auroc: 0.8106
Epoch 132/250
29376/29376 [==============================] - 2s 75us/sample - loss: 0.5248 - acc: 0.7288 - auroc: 0.8104
Epoch 133/250
29376/29376 [==============================] - 2s 73us/sample - loss: 0.5256 - acc: 0.7302 - auroc: 0.8107
Epoch 134/250
29376/29376 [==============================] - 2s 70us/sample - loss: 0.5234 - acc: 0.7296 - auroc: 0.8113
Epoch 135/250
29376/29376 [==============================] - 2s 74us/sample - loss: 0.5232 - acc: 0.7324 - auroc: 0.8100
Epoch 136/250
29376/29376 [==============================] - 3s 86us/sample - loss: 0.5234 - acc: 0.7289 - auroc: 0.8127
Epoch 137/250
29376/29376 [==============================] - 2s 65us/sample - loss: 0.5219 - acc: 0.7322 - auroc: 0.8122
Epoch 138/250
29376/29376 [=====

29376/29376 [==============================] - 2s 85us/sample - loss: 0.4974 - acc: 0.7468 - auroc: 0.8322
Epoch 198/250
29376/29376 [==============================] - 2s 69us/sample - loss: 0.4978 - acc: 0.7470 - auroc: 0.8327
Epoch 199/250
29376/29376 [==============================] - 2s 63us/sample - loss: 0.4964 - acc: 0.7488 - auroc: 0.8334
Epoch 200/250
29376/29376 [==============================] - 2s 63us/sample - loss: 0.4959 - acc: 0.7488 - auroc: 0.8340
Epoch 201/250
29376/29376 [==============================] - 2s 68us/sample - loss: 0.4968 - acc: 0.7483 - auroc: 0.8327
Epoch 202/250
29376/29376 [==============================] - 2s 63us/sample - loss: 0.4968 - acc: 0.7502 - auroc: 0.8341
Epoch 203/250
29376/29376 [==============================] - 2s 66us/sample - loss: 0.4957 - acc: 0.7488 - auroc: 0.8341
Epoch 204/250
29376/29376 [==============================] - 2s 66us/sample - loss: 0.4953 - acc: 0.7499 - auroc: 0.8328
Epoch 205/250
29376/29376 [===================

In [26]:
pred= model.predict(X_validation_set)


Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.


In [27]:
nn_pred_test = model.predict(test_transformed)
nn_pred_test_conv = [y[0] for y in nn_pred_test]


In [28]:
submission_nn = application_test[['SK_ID_CURR']]
submission_nn['TARGET'] = nn_pred_test_conv

<ipython-input-28-30b7dc9e3cc9>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_nn['TARGET'] = nn_pred_test_conv


## Observation : With NN we can achieve 0.85 ROC-AUC score after 250 echoes also it has good accuracy

## 7.Hyper Parameter Tuning 

## 7.1 Logistic Regression - Grid Search CV

In [29]:
##Tuning C Value
logistic_regressor = LogisticRegression()

c_space = np.logspace(0.01,0.01,1) 
param_grid = {'C': c_space} 

logreg_cv = GridSearchCV(logistic_regressor, param_grid, cv = 5) 
logreg_cv.fit(X_training_set,y_training_set)


log_regression_pred = logreg_cv.predict(X_validation_set)

C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regres

In [30]:
metric_calc(log_regression_pred)

The accuracy score :  0.6710208030962748
The classification report is as follows:
               precision    recall  f1-score   support

           0       0.65      0.61      0.63      6578
           1       0.69      0.73      0.71      7891

    accuracy                           0.67     14469
   macro avg       0.67      0.67      0.67     14469
weighted avg       0.67      0.67      0.67     14469

Taregt Values:
1    8323
0    6146
Name: target, dtype: int64
ROC AUC score is:  0.6655573493192576


In [31]:
log_regression_pred_test = logreg_cv.predict_proba(test_transformed)
log_regression_pred_test[:,1]
submission_log_regression = application_test[['SK_ID_CURR']]
submission_log_regression['TARGET'] = log_regression_pred_test[:,1]

<ipython-input-31-958f2f9d48fc>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  submission_log_regression['TARGET'] = log_regression_pred_test[:,1]


In [32]:
print("Best score is {}".format(logreg_cv.best_score_)) 


Best score is 0.6668709123299972


## Observation : ROC-AUC Score not much improved after tuning the C value - let's try with XG Boost Tuning

## 7.2 XG Boost - BayesSearchCV

In [33]:
# Classifier
bayes_cv_tuner = BayesSearchCV(
    estimator = XGBClassifier(
        n_jobs = 1,
        objective = 'binary:logistic',
        eval_metric = 'auc',
        silent=1,
        tree_method='approx'
    ),
    search_spaces = {
        'learning_rate': (0.01, 1.0, 'log-uniform'),
        'min_child_weight': (0, 10),
        'max_depth': (0, 50),
        'max_delta_step': (0, 20),
        'subsample': (0.01, 1.0, 'uniform'),
        'colsample_bytree': (0.01, 1.0, 'uniform'),
        'colsample_bylevel': (0.01, 1.0, 'uniform'),
        'reg_lambda': (1e-9, 1000, 'log-uniform'),
        'reg_alpha': (1e-9, 1.0, 'log-uniform'),
        'gamma': (1e-9, 0.5, 'log-uniform'),
        'min_child_weight': (0, 5),
        'n_estimators': (50, 100),
        'scale_pos_weight': (1e-6, 500, 'log-uniform')
    },    
    scoring = 'roc_auc',
    cv = StratifiedKFold(
        n_splits=4,
        shuffle=True,
        random_state=42
    ),
    n_jobs = 3,
    n_iter = 8,   
    verbose = 0,
    refit = True,
    random_state = 42
)

def status_print(optim_result):
    """Status callback durring bayesian hyperparameter search"""
    
    # Get all the models tested so far in DataFrame format
    all_models = pd.DataFrame(bayes_cv_tuner.cv_results_)    
    
    # Get current parameters and the best parameters    
    best_params = pd.Series(bayes_cv_tuner.best_params_)
    print('Model #{}\nBest ROC-AUC: {}\nBest params: {}\n'.format(
        len(all_models),
        np.round(bayes_cv_tuner.best_score_, 4),
        bayes_cv_tuner.best_params_
    ))

In [34]:
result = bayes_cv_tuner.fit(train_transformed, new_application_train['TARGET'].values, callback=status_print)

Model #1
Best ROC-AUC: 0.5372
Best params: OrderedDict([('colsample_bylevel', 0.4160029192647807), ('colsample_bytree', 0.7304484857455519), ('gamma', 0.13031389926541354), ('learning_rate', 0.042815319280763466), ('max_delta_step', 13), ('max_depth', 21), ('min_child_weight', 2), ('n_estimators', 87), ('reg_alpha', 5.497557739289786e-07), ('reg_lambda', 648), ('scale_pos_weight', 275), ('subsample', 0.13556548021189216)])

Model #2
Best ROC-AUC: 0.7363
Best params: OrderedDict([('colsample_bylevel', 0.8390144719977516), ('colsample_bytree', 0.8844821246070537), ('gamma', 4.358684608480795e-07), ('learning_rate', 0.7988179462781242), ('max_delta_step', 17), ('max_depth', 3), ('min_child_weight', 1), ('n_estimators', 68), ('reg_alpha', 0.0005266983003701547), ('reg_lambda', 953), ('scale_pos_weight', 315), ('subsample', 0.9923710598637134)])

Model #3
Best ROC-AUC: 0.7363
Best params: OrderedDict([('colsample_bylevel', 0.8390144719977516), ('colsample_bytree', 0.8844821246070537), ('gam

In [35]:
# kfold cross validation of the XGBoost model

# XGB parameters found by Bayesian optimization above
model = XGBClassifier(
            objective ='binary:logistic',
            colsample_bylevel= 0.8015579071911014, 
            colsample_bytree= 0.44364889457651413, 
            gamma= 3.811128976537413e-05, 
            learning_rate= 0.270039020618534, 
            max_delta_step= 18, 
            max_depth=36, 
            min_child_weight= 2, 
            n_estimators= 83, 
            reg_alpha= 1.5057560255472018e-06, 
            reg_lambda= 659, 
            scale_pos_weight= 256, 
            subsample= 0.8835665823899177)

kfold = StratifiedKFold(n_splits=8, random_state=42)
results = cross_val_score(model, train_transformed, new_application_train['TARGET'].values, cv=kfold, scoring='roc_auc')
print("roc_auc:", "%.3f" % results.mean(), "std:", "%.3f" % results.std()) 

C:\Users\kneelagandan\Anaconda3\lib\site-packages\sklearn\model_selection\_split.py:293: FutureWarning: Setting a random_state has no effect since shuffle is False. This will raise an error in 0.24. You should leave random_state to its default (None), or set shuffle=True.
  warnings.warn(


roc_auc: 0.740 std: 0.005


## Observation : After Hyper Tuning XGB ROC-AUC Score improved from 0.666 to 0.740